## DWI Processing

In [ ]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim
from nipype.interfaces.fsl.epi import ApplyTOPUP, TOPUP, Eddy
from nipype.interfaces.fsl.dti import DTIFit
from nipype.interfaces.fsl import Merge, FLIRT
from nipype.interfaces.freesurfer import Binarize
from pandas import DataFrame, Series

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
studyhome = '/Users/catcamacho/Box/SEA'
raw_dir = studyhome + '/dti_img/raw'
output_dir = studyhome + '/dti_img/proc'
workflow_dir = studyhome + '/dti_img/workflows'
template = studyhome + '/rest_proc/templates/neonate_T2w_template_2mm.nii.gz'
rois = studyhome + '/rest_proc/neonate_seeds/rois.nii.gz'

subjects_list = ['Newborn']

proc_cores = 2 # number of cores of processing for the workflows
acq_file = studyhome + '/dti_img/misc/acq_params.txt'
index_file = studyhome + '/dti_img/misc/index.txt'

In [ ]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subjects_list)

# Datasink- where our select outputs will go
substitutions = [('_subject_id_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

### Unwarping and preprocessing

In [ ]:
## File handling nodes

pes_template={'PE_vol': raw_dir + '/%s/dti_pe*.nii.gz'}
pes_selectfiles = Node(DataGrabber(sort_filelist=True,
                                   template = raw_dir + '/%s/dti_pe*.nii.gz',
                                   field_template = pes_template,
                                   base_directory=raw_dir,
                                   infields=['subject_id'], 
                                   template_args={'PE_vol':[['subject_id']]}), 
                       name='pes_selectfiles')

dwis_template={'dwi': raw_dir + '/{subject_id}/dti_pe0_ms103.nii.gz',
               'bvec': raw_dir + '/{subject_id}/dti_pe0_ms103.bvec', 
               'bval': raw_dir + '/{subject_id}/dti_pe0_ms103.bval'}
dwis_selectfiles = Node(SelectFiles(dwis_template), name='dwis_selectfiles')

In [ ]:
## Unwarping nodes

# include only the first volume of each PE volume
trim_PEs = MapNode(Trim(end_index=1),name='trim_PEs',iterfield=['in_file'])

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = Node(Merge(dimension='t',
                       merged_file='merged_pes.nii.gz'),
                 name='merge_pes')

topup = Node(TOPUP(encoding_file=acq_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[1], encoding_file=acq_file,
                              method='jac', out_corrected='dti_unwarped.nii.gz'),
                   name='apply_topup')

trim_b0 = Node(Trim(end_index=1), name='trim_b0')

coreg_template = Node(FLIRT(out_matrix_file='xform.mat', in_file=template, out_file='warped_template.nii.gz'), name='coreg_template')

xfm_rois = Node(FLIRT(apply_xfm=True, in_file=rois, out_file='warped_rois.nii.gz',interp='nearestneighbour'), name='xfm_rois')

In [ ]:
unwarpflow = Workflow(name='unwarpflow')
unwarpflow.connect([(infosource,pes_selectfiles, [('subject_id','subject_id')]),
                    (infosource,dwis_selectfiles, [('subject_id','subject_id')]),
                    (pes_selectfiles, trim_PEs, [('PE_vol','in_file')]), 
                    (trim_PEs, merge_pes, [('out_file','in_files')]), 
                    (merge_pes, topup, [('merged_file','in_file')]), 
                    (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                          ('out_movpar','in_topup_movpar')]),
                    (dwis_selectfiles, apply_topup, [('dwi','in_files')]),
                    (apply_topup, trim_b0, [('out_corrected','in_file')]),
                    (trim_b0, coreg_template, [('out_file','reference')]),
                    (trim_b0, xfm_rois, [('out_file','reference')]),
                    (coreg_template, xfm_rois, [('out_matrix_file','in_matrix_file')]),
                    
                    (coreg_template, datasink, [('out_file','warped_template')]),
                    (xfm_rois, datasink, [('out_file','warped_ROIs')]),
                    (apply_topup, datasink, [('out_corrected','unwarped_dwi')]),
                   ])

unwarpflow.base_dir = workflow_dir
#unwarpflow.write_graph(graph2use='flat')
unwarpflow.run('MultiProc', plugin_args={'n_procs': 2, 'memory_gb':10})